In [19]:
from pathlib import Path
from itertools import combinations
from tqdm import tqdm
import os
import random
import biodivine_aeon as ba
import pickle
import math


MODELS_FOLDER = ".\\models"
SELECTED_MODELS_EXP_1_FOLDER = "\\selected_models_exp_1"
CARDINALITIES_DICT_PICKL_FILE = 'cardinalities.pickle'
MAX_COMBINATIONS_NUM = 5000
file_cache = {}
RANDOM_SEED = 10
aeon_files = [f for f in os.listdir(MODELS_FOLDER)]


def load_boolean_network(file):
    if file not in file_cache.keys():
        file_cache[file] = Path(os.path.join(MODELS_FOLDER, file)).read_text()
    return ba.BooleanNetwork.from_aeon(file_cache[file])


def get_model_colours(file, variable_combination):
    tmp_model = load_boolean_network(file)
    for v in variable_combination:
        # Remove variable's update function; make it unknown
        tmp_model.set_update_function(v, None)

    graph = ba.PerturbationGraph(tmp_model)
    # The total colour of perturbed model
    full_colours = graph.as_original().unit_colors().cardinality()
    # 2^#vars colours come from artificial perturbation parameters (is_var_pertubred)
    colours_from_variables = 2 ** len(graph.as_original().network().variables())
    # The remaining colours come from model's own uncertainty
    colours_from_params = full_colours / colours_from_variables
    return colours_from_params

In [ ]:
cardinalities_dict = {RANDOM_SEED}

for f in aeon_files:
    print(f)
    model = load_boolean_network(f)
    vars = model.variables()
    rg = model.graph()

    # Variables with 0 arity have low effect, variables over 4 could explode colours too much
    relevant_vars = [model.get_variable_name(v) for v in vars if 0 < len(rg.regulators(v)) <= 4]

    # Generate possible combinations of relevant variables
    all_combinations = [x for x in combinations(relevant_vars, 4)]

    # Make random determinisitic
    random.seed()

    # If there are too many combinations, select just a sample
    combinations_to_try = all_combinations if len(all_combinations) <= MAX_COMBINATIONS_NUM else random.sample(all_combinations, MAX_COMBINATIONS_NUM)

    # For every model, remember a list of colours cardinalities along with the variable combinations which cause them
    cardinalities_dict[f] = list(tqdm((vc, get_model_colours(f, vc) for vc in combinations_to_try), total=len(combinations_to_try)))

In [2]:
# Preserve results of long-running computation

with open(CARDINALITIES_DICT_PICKL_FILE, 'wb') as handle:
    pickle.dump(cardinalities_dict, handle)

In [7]:
# Load results of long-running computation

with open(CARDINALITIES_DICT_PICKL_FILE, 'rb') as handle:
    cardinalities_dict = pickle.load(handle)

In [8]:
print([y for _x,y in cardinalities_dict['myeloid_witness.aeon']])

[21772800.0, 87091200.0, 406425600.0, 406425600.0, 87091200.0, 406425600.0, 87091200.0, 87091200.0, 1959552.0, 9144576.0, 9144576.0, 1959552.0, 9144576.0, 1959552.0, 1959552.0, 36578304.0, 36578304.0, 7838208.0, 36578304.0, 7838208.0, 7838208.0, 170698752.0, 36578304.0, 170698752.0, 36578304.0, 36578304.0, 36578304.0, 170698752.0, 36578304.0, 36578304.0, 36578304.0, 7838208.0, 7838208.0, 36578304.0, 36578304.0, 7838208.0, 21772800.0, 101606400.0, 101606400.0, 21772800.0, 101606400.0, 21772800.0, 21772800.0, 406425600.0, 406425600.0, 87091200.0, 406425600.0, 87091200.0, 87091200.0, 1896652800.0, 406425600.0, 1896652800.0, 406425600.0, 406425600.0, 406425600.0, 1896652800.0, 406425600.0, 406425600.0, 406425600.0, 87091200.0, 87091200.0, 406425600.0, 406425600.0, 87091200.0, 9144576.0, 9144576.0, 1959552.0, 9144576.0, 1959552.0, 1959552.0, 42674688.0, 9144576.0, 42674688.0, 9144576.0, 9144576.0, 9144576.0, 42674688.0, 9144576.0, 9144576.0, 9144576.0, 1959552.0, 1959552.0, 9144576.0, 91445

In [17]:
sets = []
for _key, value in cardinalities_dict.items():
    sets.append(set([y for _x,y in value]))

available_colours_cardinality = set.intersection(*sets)
print(available_colours_cardinality)

{4665600.0, 18662400.0, 419904.0}


In [14]:
for v in available_colours_cardinality:
    print(f'Colours from params {v:.1f} (2^{math.log2(v)})')
    for m in aeon_files:
        satisfactory_var_combinations = [vc for vc, cols in cardinalities_dict[m] if cols == v]
        print(m, len(satisfactory_var_combinations))

Colours from params 4665600.0 (2^22.153631194101663)
cardiac_witness.aeon 36
erbb_witness.aeon 250
mapk_witness.aeon 361
myeloid_witness.aeon 10
tumour_witness.aeon 24
Colours from params 18662400.0 (2^24.153631194101663)
cardiac_witness.aeon 3
erbb_witness.aeon 50
mapk_witness.aeon 60
myeloid_witness.aeon 10
tumour_witness.aeon 4
Colours from params 419904.0 (2^18.67970000576925)
cardiac_witness.aeon 4
erbb_witness.aeon 50
mapk_witness.aeon 129
myeloid_witness.aeon 10
tumour_witness.aeon 2


In [15]:
print(aeon_files)

['cardiac_witness.aeon', 'erbb_witness.aeon', 'mapk_witness.aeon', 'myeloid_witness.aeon', 'tumour_witness.aeon']


In [23]:
selected_colours_cardinality = min(available_colours_cardinality)

for m in aeon_files:
    satisfactory_var_combination = [vc for vc, cols in cardinalities_dict[m] if cols == selected_colours_cardinality].pop()
    # Generate BN again (we saved just the unknown var combinations)
    model = load_boolean_network(m)
    for v in satisfactory_var_combination:
        # Remove variable's update function; make it unknown
        model.set_update_function(v, None)
    with open(f".\\{SELECTED_MODELS_EXP_1_FOLDER}\\{m.split('_')[0]}_4unknown.aeon", "wb") as handle:
        handle.write(model.to_aeon().encode())

In [24]:
aeon_files

['cardiac_witness.aeon',
 'erbb_witness.aeon',
 'mapk_witness.aeon',
 'myeloid_witness.aeon',
 'tumour_witness.aeon']